In [74]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from typing import List, Tuple
from parsing_methods import *

## To-Do
* Tag Teams
* Stables
* Trainer
* Finisher
* Trademark Moves

In [75]:
# def getGeneralInfo(wrestlerID: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
#     """
#         Arguments:
#             wrestlerID: wrestlingdata index for this wrestler, minimum is 1, maximum is 30195
#
#         Returns:
#             GeneralInfo, Facts
#     """
#     A = requests.get('https://www.wrestlingdata.com/index.php?befehl=bios&wrestler=%d'%wrestlerID)
#     print(wrestlerID)
#     wrestler = BeautifulSoup(A.text, 'html.parser')
#     children = list(wrestler.find(title="General Information").parent.parent.children)
#     GeneralInfo = {c.attrs['title'] : [list(c.children)[3].text.strip('\n')] for c in wrestler.find(title="General Information").parent.parent.children if 'attrs' in c.__dict__ and 'title' in c.attrs}
#     wrestler_name = list(wrestler.find(style="width:100%;", cellpadding="4", cellspacing="2").children)[1].find(style="font-size: 14px;").text.strip('\n')
#     res = pd.DataFrame(GeneralInfo, index = [wrestlerID])
#     res['wrestler_name'] = [wrestler_name]
#     tables2 = wrestler.find(title='Facts')
#     B = pd.read_html(str(list(list(tables2.parent.parent.parent.parent.parent.children)[3].children)[1].table))
#     return res.transpose(), B[0]

In [76]:
# def get50wrestlers(top_index: int) -> List[int]:
#     """
#         Arguments:
#             top_index: the index of the 'Rankings' page, minimum is 1, maximum is 105
#
#         Returns:
#             A list of wrestlerID's corresponding to the list of wrestlers on the Rankings page with the given page number "top_index"
#     """
#     B = requests.get('https://www.wrestlingdata.com/index.php?befehl=bios&letter=2&seite=%d'%top_index)
#     wrestlerlist = BeautifulSoup(B.text, 'html.parser')
#
#     result_list = []
#
#     # I modified the parsing loop because it had issues in cases where the wrestler didn't have a hyperlink
#     # This should work?
#     for i in range(len(list(wrestlerlist.find(title="Liste der Wrestler").children)[3:])):
#         x = list(wrestlerlist.find(title="Liste der Wrestler").children)[3:][i]
#         if len(list(list(x.children)[2].children)) > 1:
#             resulting_int = int(list(list(x.children)[2].children)[1].attrs['href'][32:])
#         result_list.append(resulting_int)
#
#     return result_list

In [77]:
# def getSample(sample_indices: List[int]) -> Tuple[pd.DataFrame, pd.DataFrame]:
#     """
#         Arguments:
#             sample_indices: A list of wrestlerID's to be used as index for this sample
#
#         Returns:
#             generalInfo, allFacts
#     """
#     sample_GeneralInfo = [None for _ in range(len(sample_indices))]
#     sample_Facts = [None for _ in range(len(sample_indices))]
#     for i in range(len(sample_indices)):
#         sample_GeneralInfo[i], sample_Facts[i] = getGeneralInfo(sample_indices[i])
#     generalInfo = pd.concat([x.transpose() for x in sample_GeneralInfo])
#
#     res = pd.Index([])
#     for x in sample_Facts:
#         res = pd.concat([pd.Series(res), pd.Series(x[0].value_counts().index)])
#     fact_columns = res.value_counts().index
#     likely_columns = [x for x in fact_columns if len(x) < 40]
#     ts = [None for _ in generalInfo.index]
#     for i in range(len(generalInfo.index)):
#         t = sample_Facts[i].groupby(0).agg(**{"%d"%generalInfo.index[i]: (1, set)})
#         ts[i] = t.loc[t.index.intersection(likely_columns)].transpose()
#     allFacts = pd.concat(ts)
#     return generalInfo, allFacts

In [78]:
ranked_wrestlers = open('wrestler_index_ordered.txt', 'r').read().split('\n')
# Last element is a '', so this is my bad way of removing it
ranked_wrestlers.pop()
ranked_wrestlers = [int(x) for x in ranked_wrestlers]

In [79]:
def clean_up_moves(elem):
    if isinstance(elem, set):
        return len(next(iter(elem)).replace(' and ', ',').split(','))
    else:
        return 0

In [80]:
# generalInfo, allFacts = getSample(ranked_wrestlers)

# fact_counts = pd.DataFrame({col: allFacts[col].apply(lambda x : x.__len__() if type(x) == set else 0) for col in allFacts.columns})
#
# fact_counts['Trademark Moves'] = allFacts['Trademark Moves'].apply(lambda x: clean_up_moves(x))
# fact_counts['Finisher'] = allFacts['Finisher'].apply(lambda x: clean_up_moves(x))
#
# fact_counts.to_csv('Data/facts_counts_ranked.csv')
#
# generalInfo.to_csv('Data/general_info_per_wrestler_ranked.csv')
#
# print('Finished writing')

In [81]:
# wrestler_info = pd.read_csv('Data/general_info_per_wrestler_ranked.csv')
# wrestler_info.rename(columns={'Unnamed: 0': 'key'}, inplace=True)
# wrestler_info.drop(['Height', 'Weight'], axis=1, inplace=True)
# wrestler_info.astype({'Weight and Height': 'str', 'Birthplace' : 'str', 'Date of Birth': 'str', 'Debut': 'str', 'Date of Death': 'str'})

In [82]:
# fact_counts = pd.read_csv('Data/facts_counts_ranked.csv')
# fact_counts

# TODO
There's a temporary list of wrestlers. We've gotta parse them to clean the list up. Parsing will focus on the following:
- Separating height and weight (we'll use kilos and meters for easiness sake)
- Separating country of birth
- Parsing date columns into datetime format

In [83]:
# This adds the country of birth as a column to the dataframe
# birth_countries = wrestler_info[~wrestler_info['Birthplace'].isnull()]['Birthplace'].apply(lambda st: st[st.find("(")+1:st.find(")")])
#
# wrestler_info['Birth Country'] = birth_countries

In [84]:
# height_and_weight = wrestler_info[~wrestler_info['Weight and Height'].isnull()]['Weight and Height'].str.split('at', expand=True)
#
# weight = height_and_weight[~height_and_weight[0].isnull()][0].apply(lambda st: st[st.find("(")+1:st.find(")")])
# weight = pd.to_numeric(weight.str[:-3], errors='coerce')
# weight = weight[~weight.isnull()]
# wrestler_info['Weight'] = weight
#
# height = height_and_weight[~height_and_weight[1].isnull()][1].apply(lambda st: st[st.find("(")+1:st.find(")")])
# height = pd.to_numeric(height.str[:-2], errors='coerce')
# height = height[~height.isnull()]
# wrestler_info['Height'] = height
#
# wrestler_info.drop(['Weight and Height'], axis=1, inplace=True)

In [85]:
# dob = wrestler_info['Date of Birth'].str.replace(r"\(.*\)","", regex=True)
# dob = pd.to_datetime(dob, errors='coerce')
#
# debut = wrestler_info['Debut'].str.replace(r"\(.*\)","", regex=True)
# debut = pd.to_datetime(debut, errors='coerce')
#
# death = wrestler_info['Date of Death'].str.replace(r"\(.*\)","", regex=True)
# death = pd.to_datetime(dob, errors='coerce')
#
# wrestler_info['Date of Birth'] = dob
# wrestler_info['Debut'] = debut
# wrestler_info['Date of Death'] = death

In [86]:
# wrestler_info

So, the wrestlers only have ratings if they have a certain number of votes.

There's a bunch of wrestlers that are missing because of this. To fix this, I will manually parse this because suffering

In [87]:
# First thing is reading in what are the indexes that we already have
saved_indexes = ranked_wrestlers

# Now to find the indexes that haven't been parsed
all_valid_indexes = [i for i in range(1, 30196)]

missing_indexes = list(set(all_valid_indexes) - set(saved_indexes))

In [88]:
len(missing_indexes)

25213

In [89]:
# generalInfo, allFacts = getSample(missing_indexes[:4000])
#
# fact_counts = pd.DataFrame({col: allFacts[col].apply(lambda x : x.__len__() if type(x) == set else 0) for col in allFacts.columns})
#
# fact_counts['Trademark Moves'] = allFacts['Trademark Moves'].apply(lambda x: clean_up_moves(x))
# fact_counts['Finisher'] = allFacts['Finisher'].apply(lambda x: clean_up_moves(x))
#
# fact_counts.to_csv('Data/facts_counts_missing_1.csv')
#
# generalInfo.to_csv('Data/general_info_per_wrestler_missing_1.csv')
#
# print('Finished writing')

In [90]:
# generalInfo, allFacts = getSample(missing_indexes[4000:8000])
#
# fact_counts = pd.DataFrame({col: allFacts[col].apply(lambda x : x.__len__() if type(x) == set else 0) for col in allFacts.columns})
#
# fact_counts['Trademark Moves'] = allFacts['Trademark Moves'].apply(lambda x: clean_up_moves(x))
# fact_counts['Finisher'] = allFacts['Finisher'].apply(lambda x: clean_up_moves(x))
#
# fact_counts.to_csv('Data/facts_counts_missing_2.csv')
#
# generalInfo.to_csv('Data/general_info_per_wrestler_missing_2.csv')
#
# print('Finished writing')

In [91]:
# generalInfo, allFacts = getSample(missing_indexes[8000:12000])
#
# fact_counts = pd.DataFrame({col: allFacts[col].apply(lambda x : x.__len__() if type(x) == set else 0) for col in allFacts.columns})
#
# fact_counts['Trademark Moves'] = allFacts['Trademark Moves'].apply(lambda x: clean_up_moves(x))
# fact_counts['Finisher'] = allFacts['Finisher'].apply(lambda x: clean_up_moves(x))
#
# fact_counts.to_csv('Data/facts_counts_missing_3.csv')
#
# generalInfo.to_csv('Data/general_info_per_wrestler_missing_3.csv')
#
# print('Finished writing')

In [92]:
# generalInfo, allFacts = getSample(missing_indexes[12000:16000])

# fact_counts = pd.DataFrame({col: allFacts[col].apply(lambda x : x.__len__() if type(x) == set else 0) for col in allFacts.columns})

# fact_counts['Trademark Moves'] = allFacts['Trademark Moves'].apply(lambda x: clean_up_moves(x))
# fact_counts['Finisher'] = allFacts['Finisher'].apply(lambda x: clean_up_moves(x))

# fact_counts.to_csv('Data/facts_counts_missing_4.csv')

# generalInfo.to_csv('Data/general_info_per_wrestler_missing_4.csv')

# print('Finished writing')

In [93]:
# generalInfo, allFacts = getSample(missing_indexes[16000:20000])

# fact_counts = pd.DataFrame({col: allFacts[col].apply(lambda x : x.__len__() if type(x) == set else 0) for col in allFacts.columns})

# fact_counts['Trademark Moves'] = allFacts['Trademark Moves'].apply(lambda x: clean_up_moves(x))
# fact_counts['Finisher'] = allFacts['Finisher'].apply(lambda x: clean_up_moves(x))

# fact_counts.to_csv('Data/facts_counts_missing_5.csv')

# generalInfo.to_csv('Data/general_info_per_wrestler_missing_5.csv')

# print('Finished writing')

In [94]:
# generalInfo, allFacts = getSample(missing_indexes[20000:22000])

# fact_counts = pd.DataFrame({col: allFacts[col].apply(lambda x : x.__len__() if type(x) == set else 0) for col in allFacts.columns})

# fact_counts['Trademark Moves'] = allFacts['Trademark Moves'].apply(lambda x: clean_up_moves(x))
# fact_counts['Finisher'] = allFacts['Finisher'].apply(lambda x: clean_up_moves(x))

# fact_counts.to_csv('Data/facts_counts_missing_6.csv')

# generalInfo.to_csv('Data/general_info_per_wrestler_missing_6.csv')

# print('Finished writing')

In [95]:
# generalInfo, allFacts = getSample(missing_indexes[22000:24000])

# fact_counts = pd.DataFrame({col: allFacts[col].apply(lambda x : x.__len__() if type(x) == set else 0) for col in allFacts.columns})

# fact_counts['Trademark Moves'] = allFacts['Trademark Moves'].apply(lambda x: clean_up_moves(x))
# fact_counts['Finisher'] = allFacts['Finisher'].apply(lambda x: clean_up_moves(x))

# fact_counts.to_csv('Data/facts_counts_missing_7.csv')

# generalInfo.to_csv('Data/general_info_per_wrestler_missing_7.csv')

# print('Finished writing')

In [96]:
# generalInfo, allFacts = getSample(missing_indexes[24000:])

# fact_counts = pd.DataFrame({col: allFacts[col].apply(lambda x : x.__len__() if type(x) == set else 0) for col in allFacts.columns})

# fact_counts['Trademark Moves'] = allFacts['Trademark Moves'].apply(lambda x: clean_up_moves(x))
# fact_counts['Finisher'] = allFacts['Finisher'].apply(lambda x: clean_up_moves(x))

# fact_counts.to_csv('Data/facts_counts_missing_8.csv')

# generalInfo.to_csv('Data/general_info_per_wrestler_missing_8.csv')

# print('Finished writing')

# Joining up everything

The previous code blocks separated the data for the sake of making parsing easier

Now we've gotta go and merge them all togther into the final one!

In [97]:

wrestler_df = pd.read_csv('Data/general_info_per_wrestler_ranked.csv')
wrestler_df_1 = pd.read_csv('Data/general_info_per_wrestler_missing_1.csv')
wrestler_df_2 = pd.read_csv('Data/general_info_per_wrestler_missing_2.csv')
wrestler_df_3 = pd.read_csv('Data/general_info_per_wrestler_missing_3.csv')
wrestler_df_4 = pd.read_csv('Data/general_info_per_wrestler_missing_4.csv')
wrestler_df_5 = pd.read_csv('Data/general_info_per_wrestler_missing_5.csv')
wrestler_df_6 = pd.read_csv('Data/general_info_per_wrestler_missing_6.csv')
wrestler_df_7 = pd.read_csv('Data/general_info_per_wrestler_missing_7.csv')
wrestler_df_8 = pd.read_csv('Data/general_info_per_wrestler_missing_8.csv')

# There's a high chance I effed up on the concatenation
wrestler_merged = pd.concat([wrestler_df, wrestler_df_1, wrestler_df_2, wrestler_df_3, wrestler_df_4, wrestler_df_5, wrestler_df_6, wrestler_df_7, wrestler_df_8])

wrestler_merged.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [98]:
fact_df = pd.read_csv('Data/facts_counts_ranked.csv')
fact_df_1 = pd.read_csv('Data/facts_counts_missing_1.csv')
fact_df_2 = pd.read_csv('Data/facts_counts_missing_2.csv')
fact_df_3 = pd.read_csv('Data/facts_counts_missing_3.csv')
fact_df_4 = pd.read_csv('Data/facts_counts_missing_4.csv')
fact_df_5 = pd.read_csv('Data/facts_counts_missing_5.csv')
fact_df_6 = pd.read_csv('Data/facts_counts_missing_6.csv')
fact_df_7 = pd.read_csv('Data/facts_counts_missing_7.csv')
fact_df_8 = pd.read_csv('Data/facts_counts_missing_8.csv')

fact_merged = pd.concat([fact_df, fact_df_1, fact_df_2, fact_df_3, fact_df_4, fact_df_5, fact_df_6, fact_df_7, fact_df_8])
fact_merged.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [99]:
# This the merge of the dataframes
merged_df = wrestler_merged.merge(fact_merged, on='id', how='left')

# For some reason there are some that entries are repeated. I've checked them, the entire row is repeated
# So we'll remove them for the sake of ease
merged_df = merged_df[~merged_df.duplicated()]
merged_df = merged_df.reset_index(drop=True)

# The last rows are kind of worthless, as they're comments that seem to apply to one wrestler that were kept
# For some weird reason
# So it's things like "Owned by Billy Fox", "Nick, please tell us your email address". Stuff that doesn't make sense
merged_df = merged_df.iloc[:, :-14]

merged_df.to_csv('all_info_wrestlers.csv')